In [ ]:
!pip install botorch
!pip install brian2

# Import

In [ ]:
# --- Jupyter setup ---
%load_ext autoreload
%autoreload 2

# --- System path setup ---
import os
import sys
sys.path.append(os.getcwd())

# Verify the current directory
print("Current directory:", os.getcwd())

# --- Environment settings ---
nrn_options = "-nogui -NSTACK 100000 -NFRAME 20000"
os.environ["NEURON_MODULE_OPTIONS"] = nrn_options

# --- Core Python libraries ---
import math
import time
import warnings
from dataclasses import dataclass
from functools import partial

# --- NumPy & Matplotlib ---
import numpy as np
np.random.seed(237)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.animation as animation

# --- Skopt (Bayesian optimization) ---
from skopt import gp_minimize
from skopt.space import Real
from skopt.plots import plot_gaussian_process, plot_convergence

# --- PyTorch & GPyTorch ---
import torch
from torch import nn
from torch.quasirandom import SobolEngine

import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood

# --- BoTorch ---
from botorch import manual_seed
from botorch.acquisition import LogExpectedImprovement, qExpectedImprovement, qLogExpectedImprovement
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.models.gp_regression import SingleTaskGP as STGP
from botorch.models.kernels import InfiniteWidthBNNKernel
from botorch.models.transforms.outcome import Standardize
from botorch.optim import optimize_acqf
from botorch.optim.optimize import optimize_acqf as optimize_acqf_fn
from botorch.test_functions import Ackley
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize

# --- Brian2 ---
from brian2 import *

# --- Custom Utilities ---
from utils.brian2_utils import set_params_utils, eqs_utils, plotting_utils, obj_func_utils
from utils.bo_utils import ibnn_utils, shared_utils, turbo_utils, baseline_bo_utils

# --- Warning filters ---
warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


# Set up model

## params

In [ ]:
# populations
N = 700
N_E = int(N * 0.8)  # pyramidal neurons
N_I = int(N * 0.2)  # interneurons
f = 0.1
p = 3
C_ext = 800
DC_amp = 0

# external stimuli
# rate = 3 * Hz # in external noise
currents_to_track = ['I_syn', 'I_AMPA_ext', 'I_GABA_rec', 'I_AMPA_rec', 'I_NMDA_rec', 'I_DC1', 'I_DC2']
currents_to_plot = currents_to_track

## currents

In [ ]:
DC_input_ts = 1 * ms
sino_input_ts = 0.1 * ms

# currently set the duration of all currents to 0, so that they are not applied

DC_start_time1 = 0  # in ms
# DC_duration1 = 1000  # in ms
DC_duration1 = 0  # in ms
DC_amp1 = 0.1  # in nA
DC_amp_slope1 = 0.001  # in nA/ms

DC_start_time2 = 500  # in ms
# DC_duration2 = 2000  # in ms
DC_duration2 = 0  # in ms
DC_amp2 = -0.05  # in nA
DC_amp_slope2 = -0.001  # in nA/ms

sino_start_time1 = 0  # in ms
# sino_duration1=3000  # in ms
sino_duration1=0  # in ms
sino_amp1=0.5  # in nA
sino_freq1=30  # in Hz

sino_start_time2 = 1000  # in ms
# sino_duration2=2000  # in ms
sino_duration2=0  # in ms
sino_amp2=0.2  # in nA
sino_freq2=50  # in Hz



DC_input1 = set_params_utils.set_DC_input(DC_amp=DC_amp1, # in nA
                DC_amp_slope=DC_amp_slope1, # in nA/ms
                DC_duration=DC_duration1, # in ms
                DC_start_time=DC_start_time1, # in ms
                timestep=DC_input_ts
                )

DC_input2 = set_params_utils.set_DC_input(DC_amp=DC_amp2, # in nA
                DC_amp_slope=DC_amp_slope2, # in nA/ms
                DC_duration=DC_duration2, # in ms
                DC_start_time=DC_start_time2, # in ms
                timestep=DC_input_ts
                )

sino_input1 = set_params_utils.set_sino_input(sino_start_time=sino_start_time1, # in ms
                    sino_duration=sino_duration1, # in ms
                    sino_amp=sino_amp1, # in nA
                    sino_freq=sino_freq1, # in Hz
                    timestep=sino_input_ts
                    )

sino_input2 = set_params_utils.set_sino_input(sino_start_time=sino_start_time2, # in ms
                    sino_duration=sino_duration2, # in ms
                    sino_amp=sino_amp2, # in nA
                    sino_freq=sino_freq2, # in Hz
                    timestep=sino_input_ts
                    )

## run initial simulation

In [ ]:
start_scope()

N_sub = int(N_E * f)
N_non = int(N_E * (1. - f * p))


E_neuron_index = [0] # index of the neuron in the population
E_index_map = {0: 'nonselective'} # map the index in the monitor to population name
for i in range(p):
    E_neuron_index.append(N_non + i * N_sub)
    E_index_map[i+1] = f'selective {i}'


# voltage
V_L, V_thr, V_reset, V_E, V_I = set_params_utils.set_voltage()
# membrane capacitance and membrane leak
C_m_E, C_m_I, g_m_E, g_m_I = set_params_utils.set_membrane_params()

# AMPA (excitatory)
g_AMPA_ext_E, g_AMPA_rec_E, g_AMPA_ext_I, g_AMPA_rec_I, tau_AMPA = set_params_utils.set_AMPA_params(N_E)
# NMDA (excitatory)
g_NMDA_E, g_NMDA_I, tau_NMDA_rise, tau_NMDA_decay, alpha, Mg2 = set_params_utils.set_NMDA_params(N_E)
# GABAergic (inhibitory)
g_GABA_E, g_GABA_I, tau_GABA = set_params_utils.set_GABA_params(N_I)

# Write the equations for the target population (e.g., excitatory population P_E)
eqs_E = eqs_utils.write_eqs_E()
eqs_I = eqs_utils.write_eqs_I()
eqs_glut, eqs_pre_glut, eqs_pre_gaba = eqs_utils.write_other_eqs()

# neuron groups 
P_E, P_I = set_params_utils.set_neuron_groups(N_E, N_I, eqs_E, eqs_I, V_L)
# synapses
external_noise_rate = 3 * Hz
C_E, C_I, C_E_E, C_E_I, C_I_I, C_I_E, C_P_E, C_P_I = set_params_utils.set_synapses(P_E, P_I, N_E, N_I, N_sub, N_non, p, f, C_ext, external_noise_rate, eqs_glut, eqs_pre_glut, eqs_pre_gaba)


N_activity_plot = 15
current_monitor_E, r_E_sels, r_E, r_I = set_params_utils.set_monitors_for_optimization_algorithm(N_activity_plot, N_non, N_sub, p, P_E, P_I, E_neuron_index=E_neuron_index, currents_to_track=currents_to_track)

## set external stimuli
# at 1s, select population 1
C_selection = int(f * C_ext)
rate_selection = 25 * Hz


stimuli1 = TimedArray(np.r_[np.zeros(40), np.ones(2), np.zeros(100)], dt=25 * ms)
input1 = PoissonInput(P_E[N_non:N_non + N_sub], 's_AMPA_ext', C_selection, rate_selection, 'stimuli1(t)')

# # at 2s, select population 2
# stimuli2 = TimedArray(np.r_[np.zeros(80), np.ones(2), np.zeros(100)], dt=25 * ms)
# input2 = PoissonInput(P_E[N_non + N_sub:N_non + 2 * N_sub], 's_AMPA_ext', C_selection, rate_selection, 'stimuli2(t)')


# simulate, can be long >120s
net = Network(collect())
net.add(r_E_sels)
net.add(P_E, P_I, C_E_E, C_E_I, C_I_I, C_I_E, C_P_E, C_P_I)


net.store('initial')

net.run(5 * second, report='stdout')

plotting_utils.plot_firing_rate(r_E, r_I, r_E_sels)
plotting_utils.plot_currents(current_monitor_E, None, currents_to_plot, E_index_map)

## set obj func

In [ ]:
DC_amp1_range = [-0.3, 0]
DC_amp_slope1_range = [-0.2, 0.2]
DC_start_time1_range = [0, 4000]
DC_duration1_range = [20, 5000]


DC_amp2_range = [0, 0.3]
DC_amp_slope2_range = [-0.2, 0.2]
DC_start_time2_range = [0, 4000]
DC_duration2_range = [20, 4000]
# sino_start_time1_range = [0, 4000]
# sino_duration1_range = [20, 4000]
# sino_amp1_range = [0, 0.3]
# sino_freq1_range = [0.01, 10]

# sino_start_time2_range = [0, 4000]
# sino_duration2_range = [20, 4000]
# sino_amp2_range = [0, 0.3]
# sino_freq2_range = [0.01, 10]


space = [
            Real(DC_start_time1_range[0], DC_start_time1_range[1], name='DC_start_time1'),
            Real(DC_duration1_range[0], DC_duration1_range[1], name='DC_duration1'),
            Real(DC_amp1_range[0], DC_amp1_range[1], name='DC_amp1'),
            Real(DC_amp_slope1_range[0], DC_amp_slope1_range[1], name='DC_amp_slope1'),
            Real(DC_start_time2_range[0], DC_start_time2_range[1], name='DC_start_time2'),
            Real(DC_duration2_range[0], DC_duration2_range[1], name='DC_duration2'),
            Real(DC_amp2_range[0], DC_amp2_range[1], name='DC_amp2'),
            Real(DC_amp_slope2_range[0], DC_amp_slope2_range[1], name='DC_amp_slope2'),
            # Real(sino_start_time1_range[0], sino_start_time1_range[1], name='sino_start_time1'),
            # Real(sino_duration1_range[0], sino_duration1_range[1], name='sino_duration1'),
            # Real(sino_amp1_range[0], sino_amp1_range[1], name='sino_amp1'),
            # Real(sino_freq1_range[0], sino_freq1_range[1], name='sino_freq1'),
            # Real(sino_start_time2_range[0], sino_start_time2_range[1], name='sino_start_time2'),
            # Real(sino_duration2_range[0], sino_duration2_range[1], name='sino_duration2'),
            # Real(sino_amp2_range[0], sino_amp2_range[1], name='sino_amp2'),
            # Real(sino_freq2_range[0], sino_freq2_range[1], name='sino_freq2')
            ]


namespace = {'V_L': V_L, 'V_thr': V_thr, 'V_reset': V_reset, 'V_E': V_E, 'V_I': V_I,
            'C_m_E': C_m_E, 'C_m_I': C_m_I, 'g_m_E': g_m_E, 'g_m_I': g_m_I,
            'g_AMPA_ext_E': g_AMPA_ext_E, 'g_AMPA_rec_E': g_AMPA_rec_E, 'g_AMPA_ext_I': g_AMPA_ext_I,
            'g_AMPA_rec_I': g_AMPA_rec_I, 'tau_AMPA': tau_AMPA,
            'g_NMDA_E': g_NMDA_E, 'g_NMDA_I': g_NMDA_I,
            'tau_NMDA_rise': tau_NMDA_rise, 'tau_NMDA_decay': tau_NMDA_decay,
            'alpha': alpha, 'Mg2': Mg2,
            'g_GABA_E': g_GABA_E, 'g_GABA_I': g_GABA_I, 'tau_GABA': tau_GABA,
            'stimuli1': stimuli1, 
            #'stimuli2': stimuli2,
            }

objective_with_factor = partial(obj_func_utils.objective_function, net=net, namespace=namespace, current_monitor_E=current_monitor_E, r_E=r_E, r_I=r_I, r_E_sels=r_E_sels,
                                 E_index_map=E_index_map, maximize=False)


objective_func_tensor = partial(obj_func_utils.objective_function, net=net, namespace=namespace, current_monitor_E=current_monitor_E, r_E=r_E, r_I=r_I, r_E_sels=r_E_sels,
                                 process_input_func=obj_func_utils.process_input_ibnn, process_output_func=obj_func_utils.process_output_ibnn, E_index_map=E_index_map,
                                 maximize=True)


In [ ]:
pls_stop 
# go to BP-IBNN

## test obj func

In [ ]:
DC_start_time1 = 1000  # in ms
DC_duration1 = 300  # in ms
DC_amp1 = 0.1  # in nA
DC_amp_slope1 = 0.001  # in nA/ms

DC_start_time2 = 200  # in ms
DC_duration2 = 1000  # in ms
DC_amp2 = -0.05  # in nA
DC_amp_slope2 = -0.001  # in nA/ms

sino_start_time1 = 0  # in ms
sino_duration1=1000  # in ms
sino_amp1=0.5  # in nA
sino_freq1=20  # in Hz

sino_start_time2 = 1000  # in ms
sino_duration2=500  # in ms
sino_amp2=0.2  # in nA
sino_freq2=10  # in Hz


# make sure that we can run the below with no bugs
params = [DC_start_time1, DC_duration1, DC_amp1, DC_amp_slope1,
          DC_start_time2, DC_duration2, DC_amp2, DC_amp_slope2, 
        #   sino_start_time1, sino_duration1, sino_amp1, sino_freq1,
        #   sino_start_time2, sino_duration2, sino_amp2, sino_freq2
          ]

objective_with_factor(params)

## rerun (optional)

In [ ]:
#title_prefix = f'sino_amp2 = {sino_amp2} nA: '
title_prefix = ''
net.restore('initial')

DC_input_ts = 1 * ms
sino_input_ts = 0.1 * ms

DC_start_time1 = 0  # in ms
DC_duration1 = 1000  # in ms
DC_amp1 = 0.1  # in nA
DC_amp_slope1 = 0.001  # in nA/ms

DC_start_time2 = 500  # in ms
DC_duration2 = 2000  # in ms
DC_amp2 = -0.05  # in nA
DC_amp_slope2 = 0.001  # in nA/ms

sino_start_time1 = 0  # in ms
sino_duration1=3000  # in ms
sino_amp1=0.08  # in nA
sino_freq1=5  # in Hz

sino_start_time2 = 1000  # in ms
sino_duration2=2000  # in ms
sino_amp2=0.06  # in nA
sino_freq2=10  # in Hz


DC_input1 = set_params_utils.set_DC_input(DC_amp=DC_amp1, # in nA
            DC_duration=DC_duration1, # in ms
            DC_start_time=DC_start_time1, # in ms
            timestep=DC_input_ts
            )

DC_input2 = set_params_utils.set_DC_input(DC_amp=DC_amp2, # in nA
            DC_duration=DC_duration2, # in ms
            DC_start_time=DC_start_time2, # in ms
            timestep=DC_input_ts
            )

sino_input1 = set_params_utils.set_sino_input(sino_start_time=sino_start_time1, # in ms
                sino_duration=sino_duration1, # in ms
                sino_amp=sino_amp1, # in nA
                sino_freq=sino_freq1, # in Hz
                timestep=sino_input_ts
                )

sino_input2 = set_params_utils.set_sino_input(sino_start_time=sino_start_time2, # in ms
                sino_duration=sino_duration2, # in ms
                sino_amp=sino_amp2, # in nA
                sino_freq=sino_freq2, # in Hz
                timestep=sino_input_ts
                )



net.run(5 * second, report='stdout')
plotting_utils.plot_firing_rate(r_E, r_I, r_E_sels, title_prefix=title_prefix)
# plotting_utils.plot_currents(current_monitor_E, current_monitor_I, currents_to_plot, E_index_map, title_prefix=title_prefix)

plotting_utils.plot_currents(current_monitor_E, None, ['I_DC1', 'I_DC2'], E_index_map, title_prefix='')
plt.gcf().subplots_adjust(left=0.25)


In [ ]:
t_in_window, rate_in_window, mean_fr = obj_func_utils.calculate_fr_in_window(r_E_sels[0])

# Train all

## BO - IBNN

https://botorch.org/docs/tutorials/ibnn_bo/

Infinite-Width Bayesian Neural Networks for Bayesian Optimization

In this tutorial, we present an overview of infinite-width Bayesian neural networks (I-BNNs) [1, 2] and show how to use them as surrogate models for Bayesian optimization (BO).

Consider an fully connected neural network with $L$ hidden layers, parameter weights drawn from $\mathcal{N(0, \sigma_w)}$, bias terms drawn from $\mathcal{N(0, \sigma_b)}$, and nonlinearity $\phi$. In the infinite-width limit, the output of this network is exactly equivalent to $\mathcal{GP}(\mu, K^L)$. By the central limit theorem, we find $\mu(x) = 0$, and we can also recursively define the covariance function as
$$K^0(x, x')=\sigma_b^2+\sigma_w^2\frac{x \cdot x'}{d_\text{input}}\qquad K^l(x, x')=\sigma_b^2+\sigma_w^2F_\phi(K^{l-1}(x, x'), K^{l-1}(x, x), K^{l-1}(x', x'))$$
where $F_\phi$ is a deterministic function based on the activation function $\phi$.

We will refer to this kernel as the "I-BNN kernel". Unlike many popular GP kernels, I-BNN covariance function is not based on Euclidean distance, allowing the GP to represent nonstationary functions. This is advantageous for many settings of Bayesian optimization, since the function we want to optimize may not have similar behavior throughout the entire input space. Furthermore, I-BNNs have been shown to work particularly well for BO problems with high-dimensional inputs [3].

BoTorch has an implementation of I-BNNs with ReLU activations: `InfiniteWidthBNNKernel`.


[1] [Y. Cho, and L. Saul. Kernel Methods for Deep Learning. Advances in Neural Information Processing Systems 22, 2009.](https://papers.nips.cc/paper_files/paper/2009/hash/5751ec3e9a4feab575962e78e006250d-Abstract.html)  
[2] [J. Lee, Y. Bahri, R. Novak, S. Schoenholz, J. Pennington, and J. Dickstein. Deep Neural Networks as Gaussian Processes. International Conference on Learning Representations 2018.](https://arxiv.org/abs/1711.00165)  
[3] [Y.L. Li, T.G.J. Rudner, A.G. Wilson. A Study of Bayesian Neural Network Surrogates for Bayesian Optimization. International Conference on Learning Representations 2024.](https://arxiv.org/abs/2305.20028)

### import

In [ ]:
warnings.filterwarnings('ignore')

%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
tkwargs = {"device": device, "dtype": dtype}

SMOKE_TEST = os.environ.get("SMOKE_TEST")

### stim_params

In [ ]:
torch.set_printoptions(sci_mode=False)
dim = len(space)
stim_bounds = torch.tensor([[dim.low, dim.high] for dim in space], dtype=torch.float).T

In [ ]:
# params = tensor([[DC_amp1, DC_start_time1, DC_duration1, DC_amp2, DC_start_time2, DC_duration2, \
#         sino_start_time1, sino_duration1, sino_amp1, sino_freq1, \
#         sino_start_time2, sino_duration2, sino_amp2, sino_freq2]])

**Initializing the Model**: We now define two versions of the I-BNN, constructed using a GP with an `InfiniteWidthBNNKernel`. One version has fixed user-specified values for $\sigma^2_w$ and $\sigma^2_b$, and the other uses the marginal log likelihood to optimize these hyperparameters.

### get initial data

In [ ]:
acq_name = 'LogEI'
train_x, train_y = ibnn_utils.generate_initial_data(objective_func_tensor, stim_bounds, n=2)

### model

In [ ]:
INPUT_DIMS = len(space)
N_INIT = 2 if not SMOKE_TEST else 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
tkwargs = {"device": device, "dtype": dtype}

from functools import partial
# define kernels
ibnn_kernel = InfiniteWidthBNNKernel(2, device=device)
ibnn_kernel.weight_var = 10.0
ibnn_kernel.bias_var = 5.0
ibnn_kernel = ScaleKernel(ibnn_kernel, device=device)

# run BO loop
acqf_classes = {"LogEI": LogExpectedImprovement}
results = {}

In [ ]:
init_x_stored = train_x
init_y_stored = train_y

### run loop

In [ ]:
updated_x = init_x_stored
updated_y = init_y_stored

In [ ]:
optimize_hypers = True
N_ITERATIONS = 500
for acq_name, acqf_class in acqf_classes.items():
    run_bo_with_acqf = partial(ibnn_utils.gp_bo_loop, f=objective_func_tensor, bounds=stim_bounds, acqf_class=acqf_class, 
                               init_x=updated_x, 
                               init_y=updated_y, 
                               n_iterations=N_ITERATIONS,
                               )
    ibnn_x, ibnn_y = run_bo_with_acqf(kernel=ibnn_kernel, optimize_hypers=optimize_hypers)
    # matern_x, matern_y = run_bo_with_acqf(kernel=matern_kernel, optimize_hypers=True)
    # rbf_x, rbf_y = run_bo_with_acqf(kernel=rbf_kernel, optimize_hypers=True)
    results[acq_name] = {
        "BNN": (ibnn_x, ibnn_y),
        # "Matern": (matern_x, matern_y),
        # "RBF": (rbf_x, rbf_y),
    }

    acq_name = 'LogEI'
# updated_x = torch.cat([updated_x, results[acq_name]['BNN'][0]], dim=0)
# updated_y = torch.cat([updated_y, results[acq_name]['BNN'][1]], dim=0)
# # After your BO loop finishes
# torch.save(updated_x, "updated_x.pt")

In [ ]:
# updated_x = torch.cat([updated_x, results[acq_name]['BNN'][0]], dim=0)
# updated_y = torch.cat([updated_y, results[acq_name]['BNN'][1]], dim=0)

## BO with TuRBO-1 and TS/qEI

In this tutorial, we show how to implement Trust Region Bayesian Optimization (TuRBO) [1] in a closed loop in BoTorch.

This implementation uses one trust region (TuRBO-1) and supports either parallel expected improvement (qEI) or Thompson sampling (TS). We optimize the $20D$ Ackley function on the domain $[-5, 10]^{20}$ and show that TuRBO-1 outperforms qEI as well as Sobol.

Since botorch assumes a maximization problem, we will attempt to maximize $-f(x)$ to achieve $\max_x -f(x)=0$.

[1]: [Eriksson, David, et al. Scalable global optimization via local Bayesian optimization. Advances in Neural Information Processing Systems. 2019](https://proceedings.neurips.cc/paper/2019/file/6c990b7aca7bc7058f5e98ea909e924b-Paper.pdf)

Code is modified from:
https://botorch.org/docs/tutorials/baxus/

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

torch.set_printoptions(sci_mode=False)
dim = len(space)
stim_bounds = torch.tensor([[dim.low, dim.high] for dim in space], dtype=torch.float).T

### define eval_objective

In [ ]:

def eval_objective(x):
    """This is a helper function we use to unnormalize and evalaute a point"""
    return objective_func_tensor(unnormalize(x, stim_bounds))

### run loop
This simple loop runs one instance of TuRBO-1 with Thompson sampling until convergence.

TuRBO-1 is a local optimizer that can be used for a fixed evaluation budget in a multi-start fashion.  Once TuRBO converges, `state["restart_triggered"]` will be set to true and the run should be aborted.  If you want to run more evaluations with TuRBO, you simply generate a new set of initial points and then keep generating batches until convergence or when the evaluation budget has been exceeded.  It's important to note that evaluations from previous instances are discarded when TuRBO restarts.

NOTE: We use a `SingleTaskGP` with a noise constraint to keep the noise from getting too large as the problem is noise-free. 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

n_init = 2

X_turbo = turbo_utils.get_initial_points(dim, n_init, dtype=dtype, device=device)
Y_turbo = torch.tensor(
    [eval_objective(x) for x in X_turbo], dtype=dtype, device=device
).unsqueeze(-1)

init_x_stored = X_turbo
init_y_stored = Y_turbo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

batch_size = 1
max_iter= 500
max_cholesky_size = float("inf")  # Always use Cholesky

state = turbo_utils.TurboState(dim=dim, batch_size=batch_size)

NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4
N_CANDIDATES = min(5000, max(2000, 200 * dim)) if not SMOKE_TEST else 4


X_turbo, Y_turbo = turbo_utils.run_turbo_loop(X_turbo, Y_turbo, eval_objective, dim,
                   max_iter=max_iter, batch_size=batch_size,
                   max_cholesky_size=max_cholesky_size, device=device, dtype=dtype,
                   SMOKE_TEST=SMOKE_TEST,
                   result_dir='all_stored_results/turbo_results'
                   )

## baseline BO

Note: a basic version of BO is used here, but for the purpose of the final projects, we need to make changes

### run

In [ ]:
n_calls = 500
objective_with_factor = partial(obj_func_utils.objective_function, net=net, namespace=namespace, current_monitor_E=current_monitor_E, r_E=r_E, r_I=r_I, r_E_sels=r_E_sels,
                                 E_index_map=E_index_map, maximize=False)

res = gp_minimize(objective_with_factor, # the function to minimize
                  space,
                  n_initial_points= 2,
                  acq_func="EI",      # the acquisition function
                  n_calls=n_calls,         # the number of evaluations of f
                  n_random_starts=5,  # the number of random initialization points
                  noise='gaussian',       # the noise level (optional)
                  random_state=1234)   # the random seed


train_x, train_y = baseline_bo_utils.save_bo_results(res, result_dir='all_stored_results/bo_results')

### save results

In [ ]:
train_x, train_y = baseline_bo_utils.save_bo_results(res, result_dir='all_stored_results/bo_results')

In [ ]:
pls_stop

# Show best results

## IBNN

In [ ]:
most_recent_folder = shared_utils.get_most_recent_result_subfolder_name(result_dir='all_stored_results/ibnn_results/optimize_hypers', prefix = 'run_')
updated_x = torch.load(os.path.join(most_recent_folder, "updated_x.pt"))
updated_y = torch.load(os.path.join(most_recent_folder, "updated_y.pt"))

In [ ]:
shared_utils.plot_best_results(updated_x, updated_y, objective_func_tensor, num_top_points=20)

## Turbo

In [ ]:
most_recent_folder = shared_utils.get_most_recent_result_subfolder_name(result_dir='all_stored_results/turbo_results', prefix = 'run_')
# most_recent_folder = 'all_stored_results/turbo_results/temp_results'
X_turbo = torch.load(os.path.join(most_recent_folder, "updated_x.pt"))
updated_x = unnormalize(X_turbo, stim_bounds)
Y_turbo = torch.load(os.path.join(most_recent_folder, "updated_y.pt"))
updated_y = Y_turbo

In [ ]:
shared_utils.plot_best_results(updated_x, updated_y, objective_func_tensor, num_top_points=20)

## baseline BO

In [ ]:
most_recent_folder = shared_utils.get_most_recent_result_subfolder_name(result_dir='all_stored_results/bo_results', prefix = 'run_')
# most_recent_folder = 'all_stored_results/turbo_results/temp_results'
updated_x = torch.load(os.path.join(most_recent_folder, "updated_x.pt"))
updated_y = torch.load(os.path.join(most_recent_folder, "updated_y.pt"))

In [ ]:
shared_utils.plot_best_results(updated_x, updated_y, objective_func_tensor, num_top_points=20)

# Compare all three

## plot top param spread

In [ ]:
num_top_points = 20

results_dir_dict = {'baseline': 'all_stored_results/bo_results',
                    'turbo': 'all_stored_results/turbo_results',
                    'ibnn': 'all_stored_results/ibnn_results/optimize_hypers'}

x_dict, y_dict = shared_utils.load_top_optimization_results(results_dir_dict, stim_bounds, prefix='run_')

# indices_to_plot = [2, 3, 6, 7]
indices_to_plot = range(8)
shared_utils.plot_top_param_distributions(space, x_dict, y_dict, num_top_points=20, indices_to_plot=indices_to_plot)

## plot iter vs value

In [ ]:
# not cummax
fig, ax = plt.subplots(figsize=(10, 6))
for method, x_vals in x_dict.items():
    y_vals = y_dict[method]
    ax.plot(range(len(y_vals)), y_vals, label=method, alpha=0.5, linewidth=1)
ax.set_xlabel("BO Iterations")
ax.set_ylabel("Objective Value")
ax.legend()
plt.title("Objective Value Over Iterations")
plt.show()

In [ ]:
# cummax
fig, ax = plt.subplots(figsize=(10, 6))
for method, x_vals in x_dict.items():
    y_vals = y_dict[method]
    cum_max = (torch.cummax(y_vals, dim=0)[0]).cpu()
    ax.plot(range(len(cum_max)), cum_max, label=method, alpha=0.5, linewidth=1)
ax.set_xlabel("BO Iterations")
ax.set_ylabel("Max Value")
ax.legend()
plt.show()